In [1]:
import tensorflow as tf
import os

print(tf.__version__)

2.0.0


In [2]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
# # 获取物理gpu, cpu对象
# gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
# cpus = tf.config.experimental.list_physical_devices(device_type='CPU')

# # 设置当前程序的　物理可见设备范围
# tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')

# #设置仅在需要时申请：
# # for gpu in gpus:
# #     tf.config.experimental.set_memory_growth(gpu, True)

# # 设置在物理gpu上设置虚拟gpu，并用来限制gpu内存使用
# tf.config.experimental.set_virtual_device_configuration(
#     gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024), tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])

# # 获取当前虚拟gpu对象
# logical_gpus = tf.config.experimental.list_logical_devices('GPU')

# print('物理gpu个数: ', len(gpus))
# print('总的gpu个数，其中包括括虚拟Logical gpus: ', len(logical_gpus))

In [3]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6,kernel_size=5,activation='sigmoid',input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=16,kernel_size=5,activation='sigmoid'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120,activation='sigmoid'),
    tf.keras.layers.Dense(84,activation='sigmoid'),
    tf.keras.layers.Dense(10,activation='sigmoid')
])


In [4]:
X = tf.random.uniform((1,28,28,1))
for layer in net.layers:
    X = layer(X)
    print(layer.name, ',output shape\t:', X.shape)


conv2d ,output shape	: (1, 24, 24, 6)
max_pooling2d ,output shape	: (1, 12, 12, 6)
conv2d_1 ,output shape	: (1, 8, 8, 16)
max_pooling2d_1 ,output shape	: (1, 4, 4, 16)
flatten ,output shape	: (1, 256)
dense ,output shape	: (1, 120)
dense_1 ,output shape	: (1, 84)
dense_2 ,output shape	: (1, 10)


In [5]:
# 获取数据
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

def data_scale(x, y):
    x = tf.cast(x, tf.float32)
#     x = x / 255
    # 注意这里需要增加一个 通道数 维度
    x = tf.reshape(x, (x.shape[0],x.shape[1],-1))
    return x, y

train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60).map(data_scale).batch(128)
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(60).map(data_scale).batch(128)
next(iter(train_db))[0]

<tf.Tensor: id=249, shape=(128, 28, 28, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

net.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [7]:
history = net.fit_generator(generator=train_db, epochs=20, validation_data=test_db)

Epoch 1/20
469/469 [==============================] - 44s 95ms/step - loss: 0.7549 - accuracy: 0.7017 - val_loss: 0.5256 - val_accuracy: 0.8017
Epoch 2/20
469/469 [==============================] - 46s 97ms/step - loss: 0.4620 - accuracy: 0.8237 - val_loss: 0.4471 - val_accuracy: 0.8267
Epoch 3/20
469/469 [==============================] - 44s 95ms/step - loss: 0.4108 - accuracy: 0.8439 - val_loss: 0.4388 - val_accuracy: 0.8325
Epoch 4/20
469/469 [==============================] - 48s 103ms/step - loss: 0.3953 - accuracy: 0.8497 - val_loss: 0.4228 - val_accuracy: 0.8376
Epoch 5/20
469/469 [==============================] - 44s 95ms/step - loss: 0.3669 - accuracy: 0.8596 - val_loss: 0.3903 - val_accuracy: 0.8556
Epoch 6/20
469/469 [==============================] - 48s 101ms/step - loss: 0.3593 - accuracy: 0.8634 - val_loss: 0.4514 - val_accuracy: 0.8297
Epoch 7/20
469/469 [==============================] - 45s 95ms/step - loss: 0.3637 - accuracy: 0.8610 - val_loss: 0.3859 - val_accurac